In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets, model_selection
from sklearn.linear_model import LinearRegression, LogisticRegression

## Univariate linear regression with gradient descent


##### Question 1: Compute gradient of the MSE loss

The MSE loss is defined as 

$$ L(w_0, w_1) = \frac{1}{N} \sum_{n=0}^{N-1} \Big( y_n - (w_0 + w_1 x_n) \Big)^2$$

Let $u_n(w_0, w_1) = y_n - (w_0 + w_1 x_n)$ 

Then $L(w_0, w_1) = \frac{1}{N} \sum_{n=0}^{N-1} u_n(w_0, w_1)^2$

And $$\frac{\partial L}{\partial w_{0|1}} = \frac{1}{N} \sum_{n=0}^{N-1} 2 u_n \frac{ \partial u_n}{\partial w_{0|1}}$$

With $$\frac{ \partial u_n}{\partial w_{0}} = -1 $$
And $$\frac{ \partial u_n}{\partial w_{1}} = -x_n $$

Therefore
$$\frac{ \partial L}{\partial w_{0}} = - \frac{2}{N} \sum_{n=0}^{N-1} u_n(w_0, w_1)^2$$
And $$\frac{ \partial L}{\partial w_{1}} = - \frac{2}{N} \sum_{n=0}^{N-1} x_n u_n(w_0, w_1)^2$$

In [ ]:
# Read the X and y values from unilinear.csv
datapoints = np.genfromtxt('unilinear.csv', delimiter=',')
X = datapoints[:,0]
Y = datapoints[:,1]

##### Question 2

In [ ]:
    
def gradient_loss_function(X,Y,w):
    """
    Returns the gradient of the loss function defined in question 1
    """
    N = len(X)
    # partial derivative (of MSE) with respect to w0
    der_wrt_w0 = -2/N * np.sum(Y-(w[0]+w[1]*X))
    # partial derivative (of MSE) with respect to w1
    der_wrt_w1 = -2/N * np.sum(X*(Y-(w[0]+w[1]*X)))
    # make a gradient vector from the partial derivatives    
    return (np.array([der_wrt_w0,der_wrt_w1]))
    
    
def gradient_descent(
    X, 
    Y,
    weights=np.array([0, 0]),
    learning_rate=0.1,
    max_iter=1000,
):
    """
    Iteratively updates the weights using the steepest descent method
    """
    for i in range(max_iter):
        # Steepest descent algorithm formula
        weights = weights - learning_rate * gradient_loss_function(X,Y,weights)
    return weights
    

##### Question 3

In [ ]:
weights = gradient_descent(X,Y)
print(weights)

##### Question 4

In [ ]:
def closed_form_solution(X,Y):
    """
    Computes the closed form solution of the loss minimization pb
    """
    # Convert X and Y into 2D np array with one column
    # (necessary for np.matmul and np.linalg.pinv)
    x = X.reshape((-1, 1))
    x = np.c_[np.ones(len(x)), x]
    y = Y.reshape((-1, 1))
    # pinv computes the pseudo-inverse of X
    # matmul performs matrix multiplications.
    return np.matmul(np.linalg.pinv(x), y)

weights_exp = closed_form_solution(X,Y)
print(weights_exp)

##### Question 5

In [ ]:
learning_rates = [10**(-i) for i in range(1,6)]
for learning_rate in learning_rates:
    weights = gradient_descent(X,Y, learning_rate=learning_rate)
    print("Learning rate = %.5f" %learning_rate, "Weights: ", weights)

## Logistic Regression

### Logistic regression on two classes

In [ ]:
iris = datasets.load_iris()   # Load Iris dataset
print(iris["feature_names"])  # To know which column is "petal width"
print(iris["target_names"])   # To know which class is "versicolor"
X = iris.data[:, 3]           # Store the 4th feature (Petal width)    
Y = iris.target               # Store the labels

def split_wtr_versicolor(Y):
    """
    Divides the Iris dataset into two classes: Iris-Versicolor and not Iris-Versicolor
    """
    # return 1 when the class is 2 (Versicolor) otherwise return 0 (not Versicolor)
    y_bin = np.where(Y == 2, np.ones_like(Y, dtype=int), np.zeros_like(Y, dtype=int))
    return y_bin

Ybis = split_wtr_versicolor(Y)

In [ ]:
seed = 666                    # Fix random seed for reproducibility
# Shuffle and split the data into train and a concatenation of validation and test sets with a ratio of 0.7/0.3:
X_train, X_val_test, Y_train, Y_val_test = model_selection.train_test_split(X, Ybis, 
                                                                            test_size=0.3, 
                                                                            shuffle=True, 
                                                                            random_state=seed)
seed = 221
# Shuffle and split the data into validation and test sets with a ratio of 0.5/0.5:
X_val, X_test, Y_val, Y_test = model_selection.train_test_split(X_val_test, Y_val_test, 
                                                                test_size=0.5, 
                                                                shuffle=True, 
                                                                random_state=seed)                                                                            

In [ ]:
# -------------------------------------
# Train Linear regression model
# -------------------------------------
linear_regression = LinearRegression()
linear_regression.fit(X_train.reshape(-1, 1), Y_train.reshape(-1,1))

# -------------------------------------
# Train Logistic regression model
# -------------------------------------
logit = LogisticRegression()
logit.fit(X_train.reshape(-1, 1), Y_train)

# -------------------------------------
# Plot the data
# -------------------------------------
plt.figure(figsize=(10,10))
cmap = ListedColormap(['red', 'darkcyan'])  # Classes' respective colors
plt.scatter(X, Ybis, c=Ybis,
            cmap=cmap,
            label='Versicolor class')

# -------------------------------------
# Plot linear regression predictions
# -------------------------------------
# We will use this X_new instead of X_val for better plots (look more continuous)
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
plt.plot(X_new, linear_regression.predict(X_new), label="Linear regression")

# -------------------------------------
# Plot Logistic regression predictions
# -------------------------------------
Y_proba = logit.predict_proba(X_new)
plt.plot(X_new, Y_proba[:, 1], "g--", label="Logit: Iris-Versicolor Prob")
plt.plot(X_new, Y_proba[:, 0], "b--", label="Logit: not Iris-Versicolor Prob")


plt.legend(loc='best')
plt.show()

### Logistic regression on two classes

In [ ]:
print(iris["feature_names"])  # To know which columns are "petal length" and "petal width"
X = iris.data[:, :2]          # Store the 3rd (petal length) and 4th feature (Petal width)    
Y = iris.target               # Store the labels

seed = 666                    # Fix random seed for reproducibility
# Shuffle and split the data into train and a concatenation of validation and test sets with a ratio of 0.7/0.3:
X_train, X_val_test, Y_train, Y_val_test = model_selection.train_test_split(X, Y, 
                                                                            test_size=0.3, 
                                                                            shuffle=True, 
                                                                            random_state=seed)
seed = 221
# Shuffle and split the data into validation and test sets with a ratio of 0.5/0.5:
X_val, X_test, Y_val, Y_test = model_selection.train_test_split(X_val_test, Y_val_test, 
                                                                test_size=0.5, 
                                                                shuffle=True, 
                                                                random_state=seed)   

In [ ]:
### Plot parameters:
# Light colors for decision boundaries plots:
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue'])
cmap_bold = ListedColormap(['red', 'darkcyan', 'darkblue'])

def plot_iris(
    X_train,
    Y_train,
    X_val_test,
    Y_val_test,
):
    """
    Scatter plots of training and testing iris datapoints

    Colors represent specific iris species 
    Validation or test points appear in light colors
    Training points appear in bold colors
    """
    # Matplotlib method to get current axis
    ax = plt.gca()    
    # Scatter plot validation or testing points using light colors
    ax.scatter(
        X_val_test[:, 0], X_val_test[:, 1], c=Y_val_test,
        cmap=cmap_light, edgecolor='k', s=20, zorder=2
    )
    # Overlay the training points in bold colors:
    ax.scatter(
        X_train[:, 0], X_train[:, 1],c=Y_train,
        cmap=cmap_bold, edgecolor='k', s=20, zorder=2
    )
    
    plt.xlabel('Petal length')
    plt.ylabel('Petal width')
    return ax

def draw_boundaries(
    log_reg_model, 
    h=0.02,  # Step size in the mesh
):
    """
    Draw boundaries as decided by the trained model
    """
    ax = plt.gca()
    [xmin, xmax] = ax.get_xlim()
    [ymin, ymax] = ax.get_ylim()
    # Generate the axis associated to the first feature: 
    x_axis = np.arange(xmin, xmax, h)
    # Generate the axis associated to the 2nd feature: 
    y_axis = np.arange(ymin, ymax, h)
    # Generate a meshgrid (2D grid) from the 2 axis:
    x_grid, y_grid = np.meshgrid(x_axis, y_axis)
    # Vectorize the grids into column vectors:
    x_grid_vectorized = x_grid.flatten()
    x_grid_vectorized = np.expand_dims(x_grid_vectorized, axis=1)
    y_grid_vectorized = y_grid.flatten()
    y_grid_vectorized = np.expand_dims(y_grid_vectorized, axis=1)
    # Concatenate the vectorized grids:
    grid = np.concatenate((x_grid_vectorized, y_grid_vectorized),
                                  axis=1)
    # Now you can use 'grid' as data to classify by the knn 

    # Predict concatenated features to get the decision boundaries:
    decision_boundaries = log_reg_model.predict(grid)

    # Reshape the decision boundaries into a 2D matrix:
    decision_boundaries = decision_boundaries.reshape(x_grid.shape)
    plt.pcolormesh(x_grid, y_grid, decision_boundaries, cmap=cmap_light, zorder=1)
    return ax

logit = LogisticRegression()
logit.fit(X_train, Y_train)
ax = plot_iris(X_train, Y_train, X_val, Y_val)
ax = draw_boundaries(logit)